# Data Analysis Framework for Photographic Collection Data from the KHI

Introductory text

## Dataset introduction

Text content

## Table of content

Table of content

## 0. Preparation

In [7]:
!pip install pathlib
!pip install numpy
!pip install matplotlib
!pip install pandas

In [3]:
# To prevent SSL certificate failure
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

import pathlib
import xml.etree.ElementTree as ET
import re
from pprint import pprint
from resources.PhotoAttributes import PhotoAttributes
from resources.dictionaries_file import *
from resources.classes_file import *
import numpy
import matplotlib.pyplot as plt
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [4]:
%run /Users/alessandrafailla/Desktop/TESI_DHDK_2024/thesis_code/jupyter_notebook_project/resources/dataclasses_creation.py

## Upload your KHI dataset

Replace ___ with the name of your file?

### Dataset Information

## Filter photograph dataset

## Data visualization

## Download results